In [ ]:
# =============================================================================
# NAICS Eşleştirme Pipeline (Embedding + Cosine Similarity + LLM Hakemliği)
# -----------------------------------------------------------------------------
# Bu betik, muhasebe/fatura verilerini NAICS sınıfları ile eşleştirmek üzere
# üç aşamalı bir yaklaşım uygular:
#   1) Veri hazırlama ve metin temizliği
#   2) Çok dilli cümle gömme (Sentence-Transformers) + cosine similarity ile
#      aday sınıfların oluşturulması (Top-K)
#   3) LLM ile nihai seçim (hakemlik) ve açıklama üretimi
#
# Açıklama:
# - Embedding: Serbest metinleri (tedarikçi adı + ürün açıklamaları) anlamsal
#   uzayda sayısal vektörlere çevirir. Farklı diller/kısaltmalar/biçimler
#   arasındaki anlamsal benzerlikleri yakalamak için idealdir.
# - Cosine Similarity: İki vektörün yön benzerliğini ölçer [–1, +1]. 1’e
#   yakın skorlar yüksek anlamsal örtüşme demektir. Eşik (THRESHOLD) ile
#   "yeterince benzer" adayları işaretleriz.
# - LLM Hakemliği: Benzer adaylar arasında bağlama (ürün kalemlerinin özü,
#   jenerik ücretler vs.) dikkat ederek tek bir NAICS seçer ve kısa bir
#   gerekçe üretir. Embedding’in kaçırabileceği iş bağlamı sinyallerini
#   toparlar.

In [ ]:
!pip install -q sentence-transformers scikit-learn pandas openpyxl transformers torch accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 37.1 MB/s eta 0:00:00


In [ ]:
# =========================
# 1) KÜTÜPHANELER
# =========================

import os
import re
import json
import warnings
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
# ============================
# 2) MUHASEBE VERİSİ HAZIRLAMA
# ============================
# Excel dosyasını okuyup, fatura bazlı (Fiş/Fatura No + Tedarikçi) gruplama yapar ve ürün/hizmet kalemlerini aynı satırda virgülle birleştirir.
def gruplu_veri_olustur(dosya_yolu):
    print(f"'{dosya_yolu}' okunuyor...")
    df = pd.read_excel(dosya_yolu)
    df['Fiş/Fatura No'] = df['Fiş/Fatura No'].ffill()
    df['Tedarikçi / Çalışan'] = df['Tedarikçi / Çalışan'].ffill()
    print("Veri fatura bazında gruplanıyor...")
    df_grouped = df.groupby(['Fiş/Fatura No', 'Tedarikçi / Çalışan'])['Ürün/hizmet'] \
                   .apply(lambda x: ', '.join(x.dropna().astype(str))) \
                   .reset_index()
    df_grouped.columns = ['Fatura_No', 'Tedarikçi_Orjinal', 'Urunler_Orjinal']
    print("Gruplama tamamlandı.")
    return df_grouped

#TEMİZLEME FONKSİYONU
def clean_text_ultimate_pro(text): # Tedarikçi ve ürün/hizmet metinlerini normalleştirir ve gürültüyü temizler.
    text = str(text)
    text = text.replace('İ', 'i').lower()

    text = re.sub(r'\ba\.(s|ş)\b', '', text, flags=re.IGNORECASE) #aş as gibi ikili kısaltma için

    # 1. Noktaları boşlukla değiştir
    text = text.replace('.', ' ')

    # 2. Fazla boşlukları temizle
    text = re.sub(r'\s+', ' ', text)

    # 3. Türkçe karakterleri sadeleştir
    turkish_chars = {'ç': 'c', 'ğ': 'g', 'ı': 'i', 'ö': 'o', 'ş': 's', 'ü': 'u'}
    for tr, en in turkish_chars.items():
        text = text.replace(tr, en)

    # 4. Yaygın kısaltmaları kaldır
    abbreviations = [
        'as', 'ltd', 'sti', 'tic', 'tur', 'tas', 'mal', 'ins', 'amb', 've', 'san', 'limited', 'sirketi','anonim', 'sirket'
    ]
    pattern_abbr = r'\b(' + '|'.join(abbreviations) + r')\b'
    text = re.sub(pattern_abbr, '', text, flags=re.IGNORECASE)

    # 5. Ölçü birimlerini sil
    units = ['kg', 'ml', 'lt','l', 'gr', 'cl', 'mm', 'cm', 'x']
    pattern_unit = r'\d+\s*(' + '|'.join(units) + r')\b'
    text = re.sub(pattern_unit, '', text)

    # 6. Sayı, özel karakter ve son temizlik
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^,\w\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'\s*,\s*', ',', text)
    text = re.sub(r',,+', ',', text).strip(',')

    return text


# --- ANA İŞ AKIŞI ---
try:
    dosya_yolu = "/content/yeni_girdi_muhasebe.xlsx"
    df_final = gruplu_veri_olustur(dosya_yolu)

    print("Tedarikçi ve Ürünler sütunları en gelişmiş fonksiyon ile temizleniyor...")
    df_final['Tedarikçi_Temiz'] = df_final['Tedarikçi_Orjinal'].apply(clean_text_ultimate_pro)
    df_final['Urunler_Temiz'] = df_final['Urunler_Orjinal'].apply(clean_text_ultimate_pro)

    df_final['Model_Girdisi'] = df_final['Tedarikçi_Temiz'] + ', ' + df_final['Urunler_Temiz']
    df_final['Model_Girdisi'] = df_final['Model_Girdisi'].apply(lambda x: ', '.join(dict.fromkeys([i.strip() for i in re.sub(r',,+', ',', x).strip(',').split(',')])))

    print("\nİşlem Tamamlandı! Sonucun ilk 5 satırı:")

    display_cols = ['Tedarikçi_Orjinal', 'Urunler_Orjinal', 'Urunler_Temiz', 'Model_Girdisi']
    print(df_final[display_cols].head().to_string())

    output_filename = "Temizlenmis_v2.xlsx"
    df_final.to_excel(output_filename, index=False)

    print(f"\nVeri başarıyla temizlendi ve '{output_filename}' adıyla kaydedildi.")

except FileNotFoundError:
    print(f"\n!!! HATA: '{dosya_yolu}' dosyası bulunamadı.")
    print("Lütfen sol taraftaki 'Dosyalar' bölümüne dosyayı yüklediğinizden emin olun.")

'/content/yeni_girdi_muhasebe.xlsx' okunuyor...
Veri fatura bazında gruplanıyor...
Gruplama tamamlandı.
Tedarikçi ve Ürünler sütunları en gelişmiş fonksiyon ile temizleniyor...

İşlem Tamamlandı! Sonucun ilk 5 satırı:
                   Tedarikçi_Orjinal                                                                                      Urunler_Orjinal                                                                                  Urunler_Temiz                                                                                                                   Model_Girdisi
0  Turkcell Iletisim Hizmetleri A.S.  Tarife ve Paket Ücretleri, Diğer Ücretler, Gecikme Bedeli, Tahsilatına Aracılık Edilen Ödemeleriniz  tarife paket ucretleri,diger ucretler,gecikme bedeli,tahsilatina aracilik edilen odemeleriniz  turkcell iletisim hizmetleri, tarife paket ucretleri, diger ucretler, gecikme bedeli, tahsilatina aracilik edilen odemeleriniz
1  Turkcell Iletisim Hizmetleri A.S.  Tarife ve Paket Ücretl

In [ ]:
# ============================
# 3) NAICS VERİSİ HAZIRLAMA
# ============================

# NAICS metinlerini temizlemek için fonksiyon
def clean_text_for_naics(text):
    text = str(text).lower()
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^,\w\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'\s*,\s*', ',', text)
    text = re.sub(r',,+', ',', text).strip(',')
    return text

def remove_duplicate_phrases(text):
    parts = [p.strip() for p in text.split(',')]
    unique = list(dict.fromkeys(parts))  # Sıralı ve tekrarsız
    return ', '.join(unique)

try:
    # NAICS dosyasını oku
    naics_df = pd.read_excel("NAICS Codes & Descriptions.xlsx")
    print("NAICS Codes & Descriptions.xlsx başarıyla yüklendi.")

    # Boş satır olmadığından emin olmak için temel bir 'dropna' yeterli
    naics_df = naics_df.dropna(subset=['NAICS Codes', '2017 NAICS Titles (USA)', 'Index Item Descriptions'])

    # NAICS koduna göre grupla ve tüm açıklamaları birleştir
    print("NAICS kodları gruplanıyor ve açıklamalar birleştiriliyor...")
    naics_grouped = naics_df.groupby('NAICS Codes').agg(
        NAICS_Title=('2017 NAICS Titles (USA)', 'first'),
        NAICS_Item_Descriptions=('Index Item Descriptions', lambda x: ', '.join(x.astype(str)))
    ).reset_index()

    # Modele verilecek nihai birleşik açıklamayı oluştur
    naics_grouped['Combined_Description'] = naics_grouped['NAICS_Title'] + ', ' + naics_grouped['NAICS_Item_Descriptions']

    # Bu birleşik açıklamayı temizle
    print("Birleştirilmiş NAICS açıklamaları temizleniyor...")
    naics_grouped['Cleaned_Description'] = naics_grouped['Combined_Description'].apply(clean_text_for_naics)
    naics_grouped['Cleaned_Description'] = naics_grouped['Cleaned_Description'].apply(remove_duplicate_phrases)

    print("\nGruplanmış ve Temizlenmiş NAICS Verisi Örneği:")
    print(naics_grouped[['NAICS Codes', 'Cleaned_Description']].head().to_string())

    output_filename = "2_Adim_Hazir_NAICS_Verisi.xlsx"
    naics_grouped.to_excel(output_filename, index=False)

    print(f"\nNAICS verisi başarıyla hazırlandı ve '{output_filename}' adıyla kaydedildi.")

except FileNotFoundError:
    print("\n!!! HATA: 'NAICS Codes & Descriptions.xlsx' dosyası bulunamadı.")

NAICS Codes & Descriptions.xlsx başarıyla yüklendi.
NAICS kodları gruplanıyor ve açıklamalar birleştiriliyor...
Birleştirilmiş NAICS açıklamaları temizleniyor...

Gruplanmış ve Temizlenmiş NAICS Verisi Örneği:
   NAICS Codes                                                                                                                                                                                                        Cleaned_Description
0       111110                                                                                                                                                                                 soybean farming, field and seed production
1       111120  oilseed except soybean farming, canola farming, field and seed production, flaxseed farming, mustard seed farming, oilseed farming except soybean, rapeseed farming, safflower farming, sesame farming, sunflower farming
2       111130                                                                   dry pea

In [ ]:
# =========================
# 4) EŞLEŞTİRME (Embedding + LLM Hakemliği - meta-llama/Llama-3.1-8B-Instruct)
# =========================

# Parametreler
MUHASEBE_PATH = "/content/Temizlenmis_v2.xlsx"
NAICS_PATH = "/content/2_Adim_Hazir_NAICS_Verisi.xlsx"
OUT_PATH = "/content/Yeni_Final_Eslesme_LLM.xlsx"

EMBED_MODEL = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2" # çok dilli, güçlü semantik gömme
LLM_MODEL_ID = "meta-llama/Llama-3.1-8B-Instruct" # çok dilli llm modeli

HF_TOKEN = ""  #'hf_xxx' şeklinde Hugging Face token buraya koy
os.environ['HUGGING_FACE_HUB_TOKEN'] = HF_TOKEN

TOP_K = 10 # LLM'e göstereceğimiz aday sayısı (Top-10)
TOP3_K = 3 # Rapor için
THRESHOLD = 0.50

# Embedding modeli
print("Embedding modeli yükleniyor...")
embed_model = SentenceTransformer(EMBED_MODEL)

# Dosyaları yükle
df_muhasebe = pd.read_excel(MUHASEBE_PATH)
df_naics = pd.read_excel(NAICS_PATH)

# =========================
# 5) EMBEDDING VE BENZERLİK COSINE SIMILARITY
# =========================

# Cosine similaritY
# - Normdan bağımsız yönsel benzerlik ölçer. Vektör uzunluğundan etkilenmez.
# - 0.50 üzeri skorlar (pratikte) çoğu veri kümesinde anlamlı benzerlik sinyali verir.

print(" Embedding vektörleri oluşturuluyor...")
muhasebe_embeddings = embed_model.encode(df_muhasebe["Model_Girdisi"].astype(str).tolist(), show_progress_bar=True)
naics_embeddings = embed_model.encode(df_naics["Cleaned_Description"].astype(str).tolist(), show_progress_bar=True)

results = []
print(" Eşleştirme işlemi başlatıldı...")
print(" Eşleştirme işlemi başlatıldı...")
for i, query_vec in enumerate(tqdm(muhasebe_embeddings)):
    scores = util.cos_sim(query_vec, naics_embeddings)[0]  # (N,)
    n = scores.shape[0]

    # Top-10 shortlist
    k10 = min(TOP_K, n)
    top10 = torch.topk(scores, k=k10)
    top10_idx = top10.indices.tolist()
    top10_kodlar = [str(df_naics.loc[j, "NAICS Codes"]) for j in top10_idx]
    top10_kategoriler = [str(df_naics.loc[j, "NAICS_Title"]) for j in top10_idx]

    # Top-3 (rapor için)
    k3 = min(TOP3_K, n)
    top3 = torch.topk(scores, k=k3)
    top3_kodlar = [str(df_naics.loc[j, "NAICS Codes"]) for j in top3.indices.tolist()]
    top3_kategoriler = [str(df_naics.loc[j, "NAICS_Title"]) for j in top3.indices.tolist()]

    # Embedding tabanlı en iyi aday ve skor
    best_score = top10.values[0].item()
    best_idx = top10.indices[0].item()

    # Eşik mantığı:
    # - Skor >= THRESHOLD ise "embedding top-1 yeterince güçlü" kabul edilir.
    # - Aksi halde LLM nihai seçim yapacaktır
    if best_score >= THRESHOLD:
        matched_code = df_naics.loc[best_idx, "NAICS Codes"]
        matched_title = df_naics.loc[best_idx, "NAICS_Title"]
    else:
        matched_code = "Kontrol Et"
        matched_title = "Kontrol Et"

    results.append({
        "Fatura_No": df_muhasebe.loc[i, "Fatura_No"],
        "Tedarikçi_Orjinal": df_muhasebe.loc[i, "Tedarikçi_Orjinal"],
        "Urunler_Orjinal": df_muhasebe.loc[i, "Urunler_Orjinal"],
        "Atanan_NAICS_Kodu": matched_code,
        "Eşleşen_NAICS_Kategorisi": matched_title,
        "Benzerlik_Skoru": round(best_score, 4),
        "Top_3_Kodlar": ", ".join(top3_kodlar),
        "Top_3_Kategoriler": "; ".join(top3_kategoriler),
        "Top_10_Kodlar": ", ".join(top10_kodlar),
        "Top_10_Kategoriler": "; ".join(top10_kategoriler),
    })

df_results = pd.DataFrame(results)

# =========================
# 6) LLM MODELİ YÜKLEME
# =========================
# LLM, embedding ile belirlediğimiz adaylar arasından bağlama en uygun TEK sınıfı seçer.
# 4-bit quantization bellek kazanımı ve hız için
print(f"\n LLM modeli yükleniyor: {LLM_MODEL_ID}")
quant_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL_ID, token=HF_TOKEN)
llm_model = AutoModelForCausalLM.from_pretrained(
    LLM_MODEL_ID, token=HF_TOKEN, device_map="auto", quantization_config=quant_config
)
llm_pipe = pipeline("text-generation", model=llm_model, tokenizer=tokenizer, torch_dtype=torch.bfloat16)

def llm_generate_json(messages):
    # Chat modellerinde en doğrusu: apply_chat_template
    prompt_str = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    out = llm_pipe(
        prompt_str,
        max_new_tokens=200,
        temperature=0.01,
        do_sample=False,
        return_full_text=False,
    )
    return out[0]["generated_text"]


# =========================
# 7) LLM İLE NİHAİ KARAR (HAKEMLİK)
# =========================
def build_llm_prompt(row):
    codes = [c.strip() for c in str(row["Top_10_Kodlar"]).split(",") if c.strip()]
    cats  = [c.strip() for c in str(row["Top_10_Kategoriler"]).split(";") if c.strip()]

    options = []
    for i in range(len(codes)):
        cat = cats[i] if i < len(cats) else ""
        options.append(f"[{i+1}] {codes[i]} — {cat}")
    options_str = "\n".join(options)

    system_prompt = (
        "You are a meticulous accounting analyst from Turkey. "
        "Your task is to analyze an invoice's line items and choose the SINGLE most appropriate NAICS category from a given shortlist. "
        "Focus on the core service, ignoring generic fees. "
        "You MUST respond ONLY with a valid JSON object containing 'selection_index' (integer 1-10) and 'reasoning' (string, max 2 sentences in Turkish)."
    )

    user_prompt = f"""FATURA BİLGİLERİ:
- Fatura No: {row['Fatura_No']}
- Tedarikçi: {row['Tedarikçi_Orjinal']}
- Kalemler: {row['Urunler_Orjinal']}

NAICS KATEGORİ ADAYLARI:
{options_str}

Cevabını sadece istenen JSON formatında ver.
"""

    return [{"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}]

def try_parse_json(text):
    import json, re
    try:
        m = re.search(r"\{[\s\S]*\}", text)
        return json.loads(m.group(0)) if m else None
    except Exception:
        return None

llm_codes, llm_titles, llm_reasonings = [], [], []

print("\n LLM ile nihai seçimler yapılıyor...")
for _, row in df_results.iterrows():
    try:
        messages = build_llm_prompt(row)
        raw_text = llm_generate_json(messages)
        parsed = try_parse_json(raw_text)

        if parsed and "selection_index" in parsed:
            sel_idx = int(parsed["selection_index"]) - 1
            codes = [c.strip() for c in str(row["Top_10_Kodlar"]).split(",") if c.strip()]
            cats  = [c.strip() for c in str(row["Top_10_Kategoriler"]).split(";") if c.strip()]
            if 0 <= sel_idx < len(codes):
                llm_codes.append(codes[sel_idx])
                llm_titles.append(cats[sel_idx] if sel_idx < len(cats) else "")
                llm_reasonings.append(parsed.get("reasoning", ""))
                continue

        # Parse başarısızsa/indeks geçersizse: embedding top-1’e fallback
        llm_codes.append(row["Atanan_NAICS_Kodu"])
        llm_titles.append(row["Eşleşen_NAICS_Kategorisi"])
        llm_reasonings.append("LLM parse/index sorunlu, embedding top-1 ile devam.")

    except Exception as e:
        llm_codes.append(row["Atanan_NAICS_Kodu"])
        llm_titles.append(row["Eşleşen_NAICS_Kategorisi"])
        llm_reasonings.append(f"LLM error: {str(e)}")

df_results["LLM_NAICS_Kodu"] = llm_codes
df_results["LLM_NAICS_Kategorisi"] = llm_titles
df_results["LLM_Aciklama"] = llm_reasonings


# =========================
# 8) KAYDET
# =========================
df_results.to_excel(OUT_PATH, index=False)
print(f"\n Tamamlandı. Sonuçlar '{OUT_PATH}' dosyasına kaydedildi.")


🔍 Embedding modeli yükleniyor...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Device set to use cuda:0



🤖 LLM ile nihai seçimler yapılıyor...

✅ Tamamlandı. Sonuçlar '/content/Final_Eslesme_LLM.xlsx' dosyasına kaydedildi.
